<a href="https://colab.research.google.com/github/jbgh2/speech-denoising-wavenet/blob/master/Unet_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook to experiment with code

In [1]:
on_colab = True
try:
    from google.colab import drive
except:
    on_colab = False
    
print("Running on colab:", on_colab)

import os
import pathlib

if on_colab:
    drive.mount('/content/gdrive') #, force_remount=True 
    BASE_DIR = "/content/gdrive/My Drive/Colab Notebooks"
    unet_path = os.path.join(BASE_DIR, "DenoiseUNet")
else:
    BASE_DIR = os.path.join(os.getcwd())
    unet_path = BASE_DIR

print("Base dir:", BASE_DIR)

Running on colab: True
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Base dir: /content/gdrive/My Drive/Colab Notebooks


In [2]:
from zipfile import ZipFile
import soundfile as sf
import io
for s in ['clean', 'noisy']:
    for c in ['train', 'test']:
        data_dir = os.path.join(unet_path, f"data/NSDTSEA/{s}_{c}set_wav")
        output_file = os.path.join(unet_path, f"data/NSDTSEA/{s}_{c}set_wav.zip")
        #print(f"Zipping {data_dir} to {output_file}")
        #!zip -qdgds 10m -r "{output_file}" "{data_dir}"
        with ZipFile(output_file, "r") as zf:
            filenames = zf.namelist()
            print(f"{len(filenames)} files in {output_file}")

            for f in filenames[:10]:
                print(f"Load {f} with SoundFile")
                sound_bytes = zf.read(f)
                if len(sound_bytes) > 0:
                    data, sr = sf.read(io.BytesIO(sound_bytes))
                    print(len(data), sr)
                else:
                    print("Ignoring. Empty file or directory")

11573 files in /content/gdrive/My Drive/Colab Notebooks/DenoiseUNet/data/NSDTSEA/clean_trainset_wav.zip
Load content/gdrive/My Drive/Colab Notebooks/DenoiseUNet/data/NSDTSEA/clean_trainset_wav/ with SoundFile
Ignoring. Empty file or directory
Load content/gdrive/My Drive/Colab Notebooks/DenoiseUNet/data/NSDTSEA/clean_trainset_wav/p282_237.wav with SoundFile
54240 16000
Load content/gdrive/My Drive/Colab Notebooks/DenoiseUNet/data/NSDTSEA/clean_trainset_wav/p282_238.wav with SoundFile
30510 16000
Load content/gdrive/My Drive/Colab Notebooks/DenoiseUNet/data/NSDTSEA/clean_trainset_wav/p282_246.wav with SoundFile
25208 16000
Load content/gdrive/My Drive/Colab Notebooks/DenoiseUNet/data/NSDTSEA/clean_trainset_wav/p282_247.wav with SoundFile
55759 16000
Load content/gdrive/My Drive/Colab Notebooks/DenoiseUNet/data/NSDTSEA/clean_trainset_wav/p282_245.wav with SoundFile
57048 16000
Load content/gdrive/My Drive/Colab Notebooks/DenoiseUNet/data/NSDTSEA/clean_trainset_wav/p282_248.wav with Sound

In [3]:
#Test available GPU memory
show_gpu_mem = True
if on_colab and show_gpu_mem:
    # memory footprint support libraries/code
    !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
    !pip install gputil psutil humanize
    import psutil
    import humanize
    import os
    import GPUtil as GPU
    GPUs = GPU.getGPUs()
    # XXX: only one GPU on Colab and isn’t guaranteed
    gpu = GPUs[0]
    def printm():
        process = psutil.Process(os.getpid())
        print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
        print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
    printm()

Gen RAM Free: 26.2 GB  | Proc size: 165.5 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [0]:
from contextlib import contextmanager
import os

@contextmanager
def cd(newdir):
    prevdir = os.getcwd()
    os.chdir(os.path.expanduser(newdir))
    try:
        yield
    finally:
        os.chdir(prevdir)

In [5]:
if on_colab:
    print("Update repo")
    !git -C "{unet_path}" pull

    print("Install requirements")
    req_path = os.path.join(unet_path, "requirements.txt")
    print(req_path)
    !pip install -r "{req_path}"

Update repo
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 6 (delta 4), reused 6 (delta 4), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/jbgh2/speech-denoising-wavenet
   360f65d..971b906  master     -> origin/master
Updating 360f65d..971b906
Fast-forward
 models.py | 6 +++---
 1 file changed, 3 insertions(+), 3 deletions(-)
Install requirements
/content/gdrive/My Drive/Colab Notebooks/DenoiseUNet/requirements.txt


In [6]:
if on_colab:
    !pip install tensorflow-gpu==2.0.0

    import tensorflow as tf

    print("Versions:", tf.version.VERSION, tf.version.GIT_VERSION)
    print("GPU availablilty:", tf.test.is_gpu_available())
    print("Eager execution:", tf.executing_eagerly())

    gpu_devices = tf.config.experimental.list_physical_devices('GPU')
    print("GPU devices:", gpu_devices)
    if len(gpu_devices) > 0:
        tf.config.experimental.set_memory_growth(gpu_devices[0], True)
        print("Memory growth:", tf.config.experimental.get_memory_growth(gpu_devices[0]))

    #Quick test
    x = [[2.]]
    m = tf.matmul(x, x)
    print("hello, {}".format(m))

Versions: 2.0.0 v2.0.0-rc2-26-g64c3d38
GPU availablilty: True
Eager execution: True
GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Memory growth: True
hello, [[4.]]


In [7]:
if on_colab:
  print(f"Adding {unet_path} to sys.path")
  import sys
  sys.path.append(unet_path)

Adding /content/gdrive/My Drive/Colab Notebooks/DenoiseUNet to sys.path


In [0]:
from main import *

In [0]:
do_training = True

if do_training:
    command_line_args = [
                    "--mode",                   "training",
                    "--config",                 os.path.join(unet_path, "sessions/002/config.json"),
                    "--batch_size",             "4",
                    "--print_model_summary",    "True",
                    "--use_zipped_dataset"
                ]
else:
    command_line_args = [
                    "--mode",                   "inference",
                    "--config",                 os.path.join(unet_path, "sessions/001/config.json"),
                    #"--target_field_length",    "16001", 
                    "--batch_size",             "4",
                    "--noisy_input_path",       os.path.join(unet_path, "data/NSDTSEA/noisy_testset_wav"),
                    "--clean_input_path",       os.path.join(unet_path, "data/NSDTSEA/clean_testset_wav"),
                    "--print_model_summary",    "True"
                ]
set_system_settings()
cla = get_command_line_arguments(command_line_args)
config = load_config(cla.config)


In [10]:
config

{'dataset': {'extract_voice': True,
  'in_memory_percentage': 1,
  'noise_only_percent': 0.1,
  'num_condition_classes': 29,
  'path': 'data/NSDTSEA/',
  'regain': 0.06,
  'sample_rate': 16000,
  'type': 'nsdtsea'},
 'model': {'condition_encoding': 'binary',
  'dilations': 9,
  'filters': {'depths': {'final': [2048, 256], 'res': 128, 'skip': 128},
   'lengths': {'final': [3, 3], 'res': 3, 'skip': 1}},
  'input_length': 7739,
  'num_params': 6342657,
  'num_stacks': 3,
  'receptive_field_length': 6139,
  'target_field_length': 1601,
  'target_padding': 1},
 'optimizer': {'decay': 0.0,
  'epsilon': 1e-08,
  'lr': 0.001,
  'momentum': 0.9,
  'type': 'adam'},
 'training': {'batch_size': 10,
  'early_stopping_patience': 16,
  'loss': {'out_1': {'l1': 1, 'l2': 0, 'weight': 1},
   'out_2': {'l1': 1, 'l2': 0, 'weight': 1}},
  'num_epochs': 250,
  'num_test_samples': 100,
  'num_train_samples': 1000,
  'path': 'sessions/002',
  'verbosity': 2}}

In [11]:
with cd(unet_path):
    print("Working dir:", os.getcwd())
    if cla.mode == 'training':
        training(config, cla)
    elif cla.mode == 'inference':
        inference(config, cla)

Working dir: /content/gdrive/My Drive/Colab Notebooks/DenoiseUNet
Building new model...
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data_input (InputLayer)         [(None, 7739)]       0                                            
__________________________________________________________________________________________________
condition_input (InputLayer)    [(None, 5)]          0                                            
__________________________________________________________________________________________________
add_singleton_depth (AddSinglet (None, 7739, 1)      0           data_input[0][0]                 
__________________________________________________________________________________________________
initial_dense_condition (Dense) (None, 128)          640         condition_input[0][0]            
______

ValueError: ignored